## Cleaning The Data

In [79]:
import pandas as pd
import numpy as np

In [24]:
df=pd.read_csv('quikr_car.csv')
df.sample(5)

,name,company,year,Price,kms_driven,fuel_type
455,Mahindra TUV300 T4 Plus,Mahindra,2016,"5,40,000","35,000 kms",Diesel
862,Toyota Qualis,Toyota,2003,"1,80,000","1,00,000 kms",Diesel
132,Toyota Corolla,Toyota,2009,"2,75,000","26,000 kms",NaN
608,tata Indica,tata,sale,"1,30,000",NaN,NaN
405,Hyundai Sonata Transform 2.4 GDi MT,Hyundai,2017,"1,90,000","36,469 kms",Diesel


In [47]:
df.isnull().mean()*100

name          0.000000
company       0.000000
year          0.000000
Price         0.000000
kms_driven    0.000000
fuel_type     0.122399
dtype: float64

In [26]:
df.dtypes

name          object
company       object
year          object
Price         object
kms_driven    object
fuel_type     object
dtype: object

In [29]:
df['kms_driven']=df['kms_driven'].astype(str).str.split(' ').str[0].str.replace(',','')
df=df[df['kms_driven'].astype(str).str.isnumeric()]
df['kms_driven']=df['kms_driven'].astype(int)

In [35]:
df['Price']=df['Price'].astype(str).str.replace(',','')
df=df[df['Price'].astype(str).str.isnumeric()]
df['Price']=df['Price'].astype(int)

In [38]:
df['year']=df['year'].astype(int)

In [45]:
df['fuel_type'].value_counts()

fuel_type
Petrol    428
Diesel    386
LPG         2
Name: count, dtype: int64

In [49]:
mode=df['fuel_type'].mode()[0]
df['fuel_type']=df['fuel_type'].fillna(mode)

In [59]:
df['name']=df['name'].astype(str).str.split(' ').str.slice(0,3).str.join(' ')
df['name']

0         Hyundai Santro Xing
1         Mahindra Jeep CL550
3           Hyundai Grand i10
4      Ford EcoSport Titanium
6                   Ford Figo
                ...          
883        Maruti Suzuki Ritz
885            Tata Indica V2
886      Toyota Corolla Altis
888              Tata Zest XM
889        Mahindra Quanto C8
Name: name, Length: 817, dtype: object

In [62]:
df.isnull().mean()*100

name          0.0
company       0.0
year          0.0
Price         0.0
kms_driven    0.0
fuel_type     0.0
dtype: float64

In [63]:
df.describe()

,year,Price,kms_driven
count,817.000000,8.170000e+02,817.000000
mean,2012.440636,4.115503e+05,46250.714810
std,4.002354,4.749173e+05,34283.745254
min,1995.000000,3.000000e+04,0.000000
25%,2010.000000,1.750000e+05,27000.000000
50%,2013.000000,2.999990e+05,41000.000000
75%,2015.000000,4.900000e+05,56758.000000
max,2019.000000,8.500003e+06,400000.000000


In [68]:
df=df[df['Price']<5e6]

In [77]:
df=df[df['kms_driven']<190000]

In [78]:
df.describe()

,year,Price,kms_driven
count,811.000000,8.110000e+02,811.000000
mean,2012.453761,4.021844e+05,45063.913687
std,3.991196,3.815107e+05,30221.014354
min,1995.000000,3.000000e+04,0.000000
25%,2010.000000,1.750000e+05,26750.000000
50%,2013.000000,2.999990e+05,41000.000000
75%,2015.000000,4.900000e+05,56000.000000
max,2019.000000,3.100000e+06,175430.000000


## Model Training

In [81]:
x=df.drop(columns='Price')
df['Price']=np.log1p(df['Price'])
y=df['Price']

In [83]:
import optuna
import pickle
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.metrics import r2_score

In [85]:
categorical_cols = ['fuel_type','company','name']
numerical_cols = ['kms_driven', 'year']

preprocessor=ColumnTransformer(transformers=[
    ('ohe',OneHotEncoder(handle_unknown='ignore'),categorical_cols),
    ('num',StandardScaler(),numerical_cols)
])

In [86]:
def objective(trial):
    test_size = trial.suggest_float('test_size', 0.15, 0.30)
    random_state = trial.suggest_int('random_state', 0, 1000)
    model_name = trial.suggest_categorical('model', ['XGBoost', 'RandomForest', 'LinearRegression', 'LightGBM'])

    if model_name == 'XGBoost':
        model = XGBRegressor(
            n_estimators=trial.suggest_int('n_estimators', 100, 500),
            max_depth=trial.suggest_int('max_depth', 3, 15),
            learning_rate=trial.suggest_float('learning_rate', 0.01, 0.3),
            subsample=trial.suggest_float('subsample', 0.5, 1.0),
            colsample_bytree=trial.suggest_float('colsample_bytree', 0.5, 1.0),
            reg_alpha=trial.suggest_float('reg_alpha', 0, 5),
            reg_lambda=trial.suggest_float('reg_lambda', 0, 5),
            random_state=random_state,
            n_jobs=-1
        )

    elif model_name == 'RandomForest':
        model = RandomForestRegressor(
            n_estimators=trial.suggest_int('n_estimators', 100, 500),
            max_depth=trial.suggest_int('max_depth', 3, 20),
            min_samples_split=trial.suggest_int('min_samples_split', 2, 10),
            random_state=random_state,
            n_jobs=-1
        )

    elif model_name == 'LightGBM':
        model = LGBMRegressor(
            n_estimators=trial.suggest_int('n_estimators', 100, 500),
            max_depth=trial.suggest_int('max_depth', 3, 15),
            learning_rate=trial.suggest_float('learning_rate', 0.01, 0.3),
            subsample=trial.suggest_float('subsample', 0.5, 1.0),
            colsample_bytree=trial.suggest_float('colsample_bytree', 0.5, 1.0),
            reg_alpha=trial.suggest_float('reg_alpha', 0, 5),
            reg_lambda=trial.suggest_float('reg_lambda', 0, 5),
            random_state=random_state,
            n_jobs=-1
        )

    else:
        model = LinearRegression()

    pipeline = Pipeline([
        ('preprocessor', preprocessor),
        ('model', model)
    ])

    X_train_sub, X_test_sub, y_train_sub, y_test_sub = train_test_split(x, y, test_size=test_size, random_state=random_state)

    pipeline.fit(X_train_sub, y_train_sub)
    y_pred_sub = pipeline.predict(X_test_sub)
    score = r2_score(y_test_sub, y_pred_sub)

    return score

In [87]:
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=150,show_progress_bar=True)

[I 2025-07-10 19:09:06,921] A new study created in memory with name: no-name-37a74ab1-4d08-4a6a-92d4-64eec09aabf3
Best trial: 0. Best value: 0.758012:   1%|▏         | 2/150 [00:00<00:09, 14.97it/s]

[I 2025-07-10 19:09:06,953] Trial 0 finished with value: 0.7580123741213222 and parameters: {'test_size': 0.17623126621004537, 'random_state': 775, 'model': 'LinearRegression'}. Best is trial 0 with value: 0.7580123741213222.
[I 2025-07-10 19:09:07,054] Trial 1 finished with value: 0.47480708849627307 and parameters: {'test_size': 0.21638432780594208, 'random_state': 343, 'model': 'XGBoost', 'n_estimators': 344, 'max_depth': 6, 'learning_rate': 0.21373526727897507, 'subsample': 0.5412861165444609, 'colsample_bytree': 0.7355855631242694, 'reg_alpha': 2.371332080829775, 'reg_lambda': 1.7910633449030806}. Best is trial 0 with value: 0.7580123741213222.
[I 2025-07-10 19:09:07,127] Trial 2 finished with value: 0.4662151327665569 and parameters: {'test_size': 0.2113312142558985, 'random_state': 994, 'model': 'XGBoost', 'n_estimators': 365, 'max_depth': 6, 'learning_rate': 0.2639107326059824, 'subsample': 0.9259131628880357, 'colsample_bytree': 0.5555799729878976, 'reg_alpha': 2.6258752629808

c:\Users\jatin\OneDrive\Desktop\basic_project_ml\car_price\venv\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
Best trial: 0. Best value: 0.758012:   4%|▍         | 6/150 [00:00<00:09, 14.55it/s]

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] Stopped training beca

Best trial: 0. Best value: 0.758012:   5%|▌         | 8/150 [00:00<00:11, 12.39it/s]c:\Users\jatin\OneDrive\Desktop\basic_project_ml\car_price\venv\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
Best trial: 9. Best value: 0.778553:   6%|▌         | 9/150 [00:00<00:11, 12.39it/s]

...

[I 2025-07-10 19:09:16,252] Trial 137 finished with value: 0.8382991220291005 and parameters: {'test_size': 0.1633123164995904, 'random_state': 944, 'model': 'LinearRegression'}. Best is trial 133 with value: 0.8803401641685645.
[I 2025-07-10 19:09:16,286] Trial 138 finished with value: 0.8291004863941801 and parameters: {'test_size': 0.16094020995784342, 'random_state': 999, 'model': 'LinearRegression'}. Best is trial 133 with value: 0.8803401641685645.
[I 2025-07-10 19:09:16,318] Trial 139 finished with value: 0.7365479467049989 and parameters: {'test_size': 0.1744686672555651, 'random_state': 945, 'model': 'LinearRegression'}. Best is trial 133 with value: 0.8803401641685645.
[I 2025-07-10 19:09:16,352] Trial 140 finished with value: 0.8315589634069012 and parameters: {'test_size': 0.22093947609321787, 'random_state': 909, 'model': 'LinearRegression'}. Best is trial 133 with value: 0.8803401641685645.
[I 2025-07-10 19:09:16,384] Trial 141 finished with value: 0.8787015320954925 and 

Best trial: 133. Best value: 0.88034:  98%|█████████▊| 147/150 [00:09<00:00, 24.31it/s]

[I 2025-07-10 19:09:16,481] Trial 144 finished with value: 0.7847810768888182 and parameters: {'test_size': 0.15515798234775385, 'random_state': 984, 'model': 'LinearRegression'}. Best is trial 133 with value: 0.8803401641685645.
[I 2025-07-10 19:09:16,513] Trial 145 finished with value: 0.7862614890454683 and parameters: {'test_size': 0.16842719062242148, 'random_state': 951, 'model': 'LinearRegression'}. Best is trial 133 with value: 0.8803401641685645.
[I 2025-07-10 19:09:16,563] Trial 146 finished with value: 0.6840176197872657 and parameters: {'test_size': 0.15237049122846905, 'random_state': 912, 'model': 'LinearRegression'}. Best is trial 133 with value: 0.8803401641685645.


Best trial: 133. Best value: 0.88034: 100%|██████████| 150/150 [00:09<00:00, 15.09it/s]

[I 2025-07-10 19:09:16,796] Trial 147 finished with value: 0.8105046695975897 and parameters: {'test_size': 0.15953962974597494, 'random_state': 966, 'model': 'RandomForest', 'n_estimators': 130, 'max_depth': 19, 'min_samples_split': 4}. Best is trial 133 with value: 0.8803401641685645.
[I 2025-07-10 19:09:16,826] Trial 148 finished with value: 0.6752878913212851 and parameters: {'test_size': 0.16438165953876877, 'random_state': 872, 'model': 'LinearRegression'}. Best is trial 133 with value: 0.8803401641685645.
[I 2025-07-10 19:09:16,858] Trial 149 finished with value: 0.8459042586663004 and parameters: {'test_size': 0.15622798097235246, 'random_state': 930, 'model': 'LinearRegression'}. Best is trial 133 with value: 0.8803401641685645.


In [88]:
print("Best Parameters:", study.best_trial.params)
print("Best R² Score:", round(study.best_value, 4))

Best Parameters: {'test_size': 0.15892940339566472, 'random_state': 959, 'model': 'LinearRegression'}
Best R² Score: 0.8803


In [90]:
best_params = study.best_trial.params

if best_params['model'] == 'XGBoost':
    best_model = XGBRegressor(
        n_estimators=best_params['n_estimators'],
        max_depth=best_params['max_depth'],
        learning_rate=best_params['learning_rate'],
        subsample=best_params['subsample'],
        colsample_bytree=best_params['colsample_bytree'],
        reg_alpha=best_params['reg_alpha'],
        reg_lambda=best_params['reg_lambda'],
        random_state=best_params['random_state'],
        n_jobs=-1
    )
elif best_params['model'] == 'RandomForest':
    best_model = RandomForestRegressor(
        n_estimators=best_params['n_estimators'],
        max_depth=best_params['max_depth'],
        min_samples_split=best_params['min_samples_split'],
        random_state=best_params['random_state'],
        n_jobs=-1
    )
elif best_params['model'] == 'LightGBM':
    best_model = LGBMRegressor(
        n_estimators=best_params['n_estimators'],
        max_depth=best_params['max_depth'],
        learning_rate=best_params['learning_rate'],
        subsample=best_params['subsample'],
        colsample_bytree=best_params['colsample_bytree'],
        reg_alpha=best_params['reg_alpha'],
        reg_lambda=best_params['reg_lambda'],
        random_state=best_params['random_state'],
        n_jobs=-1
    )
else:
    best_model = LinearRegression()

final_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('model', best_model)
])

X_train_final, X_test_final, y_train_final, y_test_final = train_test_split(x, y, test_size=best_params['test_size'], random_state=best_params['random_state'])

final_pipeline.fit(X_train_final, y_train_final)
y_pred_final = final_pipeline.predict(X_test_final)

final_r2 = r2_score(y_test_final, y_pred_final)
print("Final R² on Test (log-scale):", round(final_r2, 4))

Final R² on Test (log-scale): 0.8803


In [92]:
with open('best_price_model.pkl', 'wb') as f:
    pickle.dump(final_pipeline, f)

print("Model saved as best_price_model.pkl")

Model saved as best_price_model.pkl
